In [1]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

In [2]:
#create global variable

background = None

accumulated_weight = 0.5

# Manually set up our ROI for grabbing the hand.
# Feel free to change these. 
roi_top = 55
roi_bottom = 300
roi_right = 350
roi_left = 600

In [3]:
def calc_accum_avg(frame,accumulated_weight):
    
    global background
    
    if background is None:
        background = frame.copy().astype('float') #input img
        return None
    
    cv2.accumulateWeighted(frame,background,accumulated_weight) 

In [4]:
def segment(frame,threshold_min=25): #bisa ganti nilai thresholdnya kalo masih ada noise
    global background
    # Calculates the Absolute Differentce between the backgroud and the passed in frame
    diff = cv2.absdiff(background.astype('uint8'),frame) #input img
    
    # Apply a threshold to the image so we can grab the foreground
    ret,thresholded = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    
    # Grab the external contours form the image
    image,contours,hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    # If length of contours list is 0, then we didn't grab any contours!
    if len(contours) == 0:
        return None
    
    else:
        # ASSUMING THE LARGEST EXTERNAL CONTOUR IN ROI, IS THE HAND
        hand_segment = max(contours,key=cv2.contourArea)
        
        return (thresholded,hand_segment)

In [5]:
def count_fingers(thresholded,hand_segment):
    
    # Calculated the convex hull of the hand segment
    conv_hull =cv2.convexHull(hand_segment)
    
    # 4 TOP POINTS (see documentation for more detail)
    top    = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left   = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right  = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    
    # In theory, the center of the hand is half way between the top and bottom and halfway between left and right
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    distance = pairwise.euclidean_distances([(cX,cY)],Y=[left,right,top,bottom])[0]
    
    # Grab the largest distance
    max_distance = distance.max()
    
    # Create a circle with 90% radius of the max euclidean distance
    radius = int(0.9*max_distance)
    circumfrence = (2*np.pi*radius)

    # Not grab an ROI of only that circle
    circular_roi = np.zeros(thresholded.shape[:2], dtype="uint8")
    
    # draw the circular ROI
    cv2.circle(circular_roi,(cX,cY),radius,255,10)
    
    # This then returns the cut out obtained using the mask on the thresholded hand image.
    circular_roi = cv2.bitwise_and(thresholded, thresholded, mask=circular_roi)
    
    # Grab contours in circle ROI
    image,contours,hierarchy = cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    count = 0
    
    for cnt in contours:
        # Increment count of fingers based on two conditions:
        (x,y,w,h) = cv2.boundingRect(cnt)
        
        # 1. Contour region is not the very bottom of hand area (the wrist)
        out_of_wrist = (cY + (cY*0.25)) > (y+h)
        
        # 2. Number of points along the contour does not exceed 25% of the circumference of the circular ROI (otherwise we're counting points off the hand)
        limit_points = ((circumfrence*0.25) > cnt.shape[0])
        
        if out_of_wrist and limit_points:
            count += 1
            
    return count

In [9]:
cam = cv2.VideoCapture(0)

# Intialize a frame count
num_frames = 0

# keep looping, until interrupted
while True:
    # get the current frame
    ret, frame = cam.read()
    
    # flip the frame so that it is not the mirror view
    frame = cv2.flip(frame, 1)
    
    # clone the frame
    frame_copy = frame.copy()
    
    # Grab the ROI from the frame
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    # Apply grayscale and blur to ROI
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    # For the first 30 frames we will calculate the average of the background.
    # We will tell the user while this is happening
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        
        if num_frames <= 59:
            cv2.putText(frame_copy,'WAIT. GETTING BACKGROUND',(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',frame_copy)
    else:
        # now that we have the background, we can segment the hand.
        
        # segment the hand region
        hand = segment(gray)
        
        # First check if we were able to actually detect a hand
        if hand is not None:
            
            # unpack
            thresholded , hand_segment = hand
            
            # DRAWS CONTOURS AROUND REAL HAND IN LIVE STREAM
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),3)
            
            # Count the fingers
            fingers = count_fingers(thresholded,hand_segment)
            
            # Display count
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            # Also display the thresholded image
            cv2.imshow('Thresholded',thresholded)
    
    # Draw ROI Rectangle on frame copy
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    
    # increment the number of frames for tracking
    num_frames += 1
    
    # Display the frame with segmented hand
    cv2.imshow('Finger Count',frame_copy)
    
    # Close windows with Esc
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()
    

## RESULT

<img src="1.PNG">
<img src="2.PNG">
<img src="3.PNG">
<img src="4.PNG">
<img src="5.PNG">